In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium import FeatureGroup
from config import vworld_key
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
#from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

In [2]:
# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])

In [3]:
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

In [4]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

### 청주시_도시재생계획구역

In [5]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/27.청주시_도시재생계획구역.geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
crp_df = pd.json_normalize(geojson_data) # city_revitalize_planning_map_df
crp_df['geometry'] = crp_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)

# 폴리곤의 중점을 찾음
centroid = crp_df['geometry'].iloc[0].centroid.buffer(0.015) # 1도의 위도 변화는 대략 111.32 킬로미터
crp_df['centroid_polygon_geometry'] = [centroid]
crp_df

,type,properties.fid,properties.구역명,geometry.type,geometry.coordinates,geometry,centroid_polygon_geometry
0,Feature,5,도시재생구역도,Polygon,"[[[127.4833417, 36.6318644], [127.4833417, 36....","POLYGON ((127.4833417 36.6318644, 127.4833417 ...",POLYGON ((127.50347965416363 36.63603755594320...


In [6]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.60720970472786, 127.63641867895493], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in crp_df.iterrows():
    popup_text = f"{row['properties.구역명']}"
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color='blue': {'fillColor': 'blue', 'color': 'black', 'weight': 1}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))  # 팝업 메시지 추가
    
m

### 격자(매핑용)

In [7]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/26.청주시_격자(매핑용).geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))
grid_map_df

,type,properties.gid,geometry.type,geometry.coordinates,geometry
0,Feature,라바122456,Polygon,"[[[127.63641867895493, 36.608165744671886], [1...",POLYGON ((127.63641867895493 36.60816574467188...
1,Feature,다바841455,Polygon,"[[[127.32221067151384, 36.60720970472786], [12...",POLYGON ((127.32221067151384 36.60720970472786...
2,Feature,라바189471,Polygon,"[[[127.71137365974974, 36.62157886605511], [12...",POLYGON ((127.71137365974974 36.62157886605511...
3,Feature,다바852482,Polygon,"[[[127.33445848271631, 36.631567839444976], [1...",POLYGON ((127.33445848271631 36.63156783944497...
4,Feature,다바959488,Polygon,"[[[127.45413724007568, 36.63708300728262], [12...",POLYGON ((127.45413724007568 36.63708300728262...
...,...,...,...,...,...
95251,Feature,다바868478,Polygon,"[[[127.35236168440159, 36.62798537617458], [12...",POLYGON ((127.35236168440159 36.62798537617458...
95252,Feature,다바974324,Polygon,"[[[127.47097166337947, 36.48924077421588], [12...",POLYGON ((127.47097166337947 36.48924077421588...
95253,Feature,라바151492,Polygon,"[[[127.66891675313607, 36.640578073348856], [1...",POLYGON ((127.66891675313607 36.64057807334885...
95254,Feature,다바841537,Polygon,"[[[127.3220406982908, 36.681132105366395], [12...",POLYGON ((127.3220406982908 36.681132105366395...


In [8]:
# crp_df에서 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
polygon_expanded = crp_df['centroid_polygon_geometry'].iloc[0]

# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')
#grid_map_df['geometry'] = GeoSeries(grid_map_df['geometry'])

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid = grid_map_df[grid_map_df['geometry'].intersects(polygon)].reset_index(drop=True) # polygon과 교차하거나 포함하는 경우를 모두 선택, within 은 포함
# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid_expanded = grid_map_df[grid_map_df['geometry'].intersects(polygon_expanded)].reset_index(drop=True) # polygon과 교차하거나 포함하는 경우를 모두 선택

In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.60720970472786, 127.63641867895493], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_grid.iterrows():
    popup_text = f"{row['properties.gid']}"
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color='black': {'fillColor': 'black', 'color': 'black', 'weight': 1}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))  # 팝업 메시지 추가
    
m

### 청주시 시간대별 유동인구

In [10]:
# 원도심 영역(도시재생 대상지역)
floating_population_df_time = pd.read_csv('SBJ_2309_001/3.청주시_시간대별_유동인구.csv')
floating_population_df_time['STD_YM'] = floating_population_df_time['STD_YM'].astype(str)
floating_population_df_time.rename(columns={'STD_YM':'연도'}, inplace=True)
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
floating_population_df_time = geo_transform(floating_population_df_time)
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
floating_population_df_time = floating_population_df_time[floating_population_df_time['geometry'].within(polygon)]

# 열 이름 변경 과정
columns_to_rename = floating_population_df_time.columns[1:-3] # 시계열 정보를 포함한 column만 선택
new_column_names = []
# 각 열 이름을 처리하여 새로운 열 이름을 생성
for column_name in columns_to_rename:
    numeric_part = column_name.split('_')[-1] # 숫자 부분 추출
    new_column_name = f"{numeric_part}시" # '시'를 붙여 새로운 열 이름 생성
    new_column_names.append(new_column_name) # new_column_names에 추가
# 열 이름을 변경합니다.
floating_population_df_time.rename(columns=dict(zip(columns_to_rename, new_column_names)), inplace=True)
floating_population_df_time

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



,연도,00시,01시,02시,03시,04시,05시,06시,07시,08시,09시,10시,11시,12시,13시,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시,lon,lat,geometry
82848,202001,1.17,1.09,1.02,0.97,0.99,1.04,1.29,1.75,2.44,2.65,2.81,3.04,3.09,3.30,3.44,3.46,3.32,3.21,2.86,2.14,1.84,1.66,1.41,1.18,127.483602,36.631043,POINT (127.48360 36.63104)
82849,202001,2.90,2.86,2.75,2.67,2.67,2.79,3.23,4.15,5.47,5.79,6.00,6.55,6.57,7.01,7.31,7.45,7.36,7.17,6.87,5.23,4.45,4.01,3.50,2.98,127.483602,36.631494,POINT (127.48360 36.63149)
82850,202001,2.88,2.81,2.70,2.63,2.61,2.75,3.23,4.29,5.97,6.16,6.23,6.67,6.73,7.19,7.42,7.52,7.38,7.26,6.97,5.30,4.52,4.08,3.57,3.00,127.483602,36.631944,POINT (127.48360 36.63194)
82851,202001,1.47,1.32,1.15,1.04,1.04,1.18,1.70,2.84,4.63,4.50,4.63,5.05,5.23,5.49,5.72,5.56,5.70,5.84,5.86,4.04,3.13,2.84,2.22,1.59,127.483602,36.632395,POINT (127.48360 36.63240)
83145,202001,1.13,1.13,0.99,0.90,0.90,0.97,1.38,2.05,3.39,3.37,3.36,3.53,3.57,3.87,4.03,3.99,3.88,3.94,3.76,2.81,2.30,2.01,1.62,1.31,127.484162,36.630592,POINT (127.48416 36.63059)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3983417,202212,0.67,0.62,0.35,0.32,0.30,0.48,1.24,2.44,4.56,4.13,3.96,4.17,4.45,4.91,4.91,4.68,4.82,5.05,5.10,3.25,2.42,2.01,1.54,0.95,127.492551,36.631945,POINT (127.49255 36.63195)
3983418,202212,3.23,2.68,1.22,1.32,1.04,1.57,3.66,9.46,17.00,15.22,15.52,16.48,17.34,19.28,19.48,19.90,21.10,22.53,22.34,15.15,12.01,10.67,7.80,4.89,127.492550,36.632396,POINT (127.49255 36.63240)
3983419,202212,1.02,1.70,0.79,0.62,0.58,0.99,1.80,3.30,6.15,5.35,5.60,5.67,5.67,6.66,6.57,6.22,6.71,6.78,6.66,4.87,3.73,3.09,2.37,1.70,127.492550,36.632847,POINT (127.49255 36.63285)
3983801,202212,1.84,1.22,1.08,0.92,1.34,2.93,7.50,12.57,16.25,14.74,16.07,19.76,21.12,21.17,19.76,19.11,20.15,21.10,18.63,11.28,7.96,6.39,4.01,2.63,127.493110,36.628339,POINT (127.49311 36.62834)


#### 유동인구 HEATMAP의 시계열 변화 시각화(시간대별)

In [11]:
# 딕셔너리 생성 및 연도 & DataFrame 매핑
filtered_floating_population_time = {}
year_list = ['2020', '2021', '2022']
for year in year_list:
    year_df = pd.DataFrame({'연도' : [year]}) # 해당 연도 df 설정(접합용)
    year_tot = pd.DataFrame() # 해당 연도 df 설정(최종용)
    floating_population_df_time_year = floating_population_df_time[floating_population_df_time['연도'].str.startswith(year)] # 특정 연도로만 필터링
    for point in floating_population_df_time_year['geometry'].unique():
        point_df = pd.DataFrame({'lon' : [point.x], 'lat' : [point.y]}) # 해당 지점 df 설정(접합용)
        fpt_point_df = floating_population_df_time_year[floating_population_df_time_year['geometry'] == point]
        
        # 1월부터 12월 까지의 월별 유동인구 평균계산 및 연도, 지점 df와 접합 & 최종 df에 접합
        column_mean = fpt_point_df.iloc[:,1:25].mean()
        column_mean_df = pd.DataFrame(column_mean).transpose()
        column_mean_df = column_mean_df.iloc[:, :24].round(2) # 소수점 2번째 자리까지만 표기
        column_mean_df = pd.concat([year_df, column_mean_df, point_df], axis=1)
        year_tot = pd.concat([year_tot, column_mean_df])
    else:
        filtered_floating_population_time[year] = year_tot

##### 2020년도 유동인구 HEATMAP 시계열 변화(시간대별)

In [ ]:
# 지도 생성
m = folium.Map(location=[36.644, 127.485], zoom_start=12)

# 데이터프레임에서 위도, 경도, 시간대별 데이터 추출
latitudes = filtered_floating_population_time['2020']['lat']
longitudes = filtered_floating_population_time['2020']['lon']

# 시간대별 히트맵 데이터 생성
heat_data = []
for i in range(24):
    time_column = f'{i:02}시'
    volume = filtered_floating_population_time['2020'][time_column]
    heat_data_hour = [[lat, lon, vol] for lat, lon, vol in zip(latitudes, longitudes, volume)]
    heat_data.append(heat_data_hour)

# HeatMapWithTime을 추가
plugins.HeatMapWithTime(heat_data, index=[f'{i:02}시' for i in range(24)], auto_play=True, max_opacity=0.6, radius=18).add_to(m)

# 지도 출력
m

##### 2021년도 유동인구 HEATMAP 시계열 변화(시간대별)

In [ ]:
# 지도 생성
m = folium.Map(location=[36.644, 127.485], zoom_start=12)

# 데이터프레임에서 위도, 경도, 시간대별 데이터 추출
latitudes = filtered_floating_population_time['2021']['lat']
longitudes = filtered_floating_population_time['2021']['lon']

# 시간대별 히트맵 데이터 생성
heat_data = []
for i in range(24):
    time_column = f'{i:02}시'
    volume = filtered_floating_population_time['2021'][time_column]
    heat_data_hour = [[lat, lon, vol] for lat, lon, vol in zip(latitudes, longitudes, volume)]
    heat_data.append(heat_data_hour)

# HeatMapWithTime을 추가
plugins.HeatMapWithTime(heat_data, index=[f'{i:02}시' for i in range(24)], auto_play=True, max_opacity=0.6, radius=18).add_to(m)

# 지도 출력
m

##### 2022년도 유동인구 HEATMAP 시계열 변화(시간대별)

In [ ]:
# 지도 생성
m = folium.Map(location=[36.644, 127.485], zoom_start=12)

# 데이터프레임에서 위도, 경도, 시간대별 데이터 추출
latitudes = filtered_floating_population_time['2022']['lat']
longitudes = filtered_floating_population_time['2022']['lon']

# 시간대별 히트맵 데이터 생성
heat_data = []
for i in range(24):
    time_column = f'{i:02}시'
    volume = filtered_floating_population_time['2022'][time_column]
    heat_data_hour = [[lat, lon, vol] for lat, lon, vol in zip(latitudes, longitudes, volume)]
    heat_data.append(heat_data_hour)

# HeatMapWithTime을 추가
plugins.HeatMapWithTime(heat_data, index=[f'{i:02}시' for i in range(24)], auto_play=True, max_opacity=0.6, radius=18).add_to(m)

# 지도 출력
m

### 청주시 성연령별 유동인구

In [19]:
# 최종저장용 df 정의 -> 각 지점들을 grid폴리곤에 매핑시켜 grid별 유동인구로 변환 -> 
filtered_floating_population_tottime = pd.DataFrame()
for grid in tqdm(filtered_grid['geometry'].unique()):
    # 격자와 매핑하기 위해 격자 내에 있는 지점으로만 필터링
    subset = floating_population_df_time[floating_population_df_time['geometry'].within(grid)]
    # 해당 구역 grid df 설정(접합용) 
    grid_df = pd.DataFrame({'geometry' : [grid]})
    # 중간저장용 df 정의
    concat_df = pd.DataFrame()
    
    for date in floating_population_df_time['연도'].unique():
        date_df = pd.DataFrame({'연도' : [date[:4]]}) # 해당 연도 df 설정(접합용) 
        subset_date = subset[subset['연도'] == date]
        # 열 별로 더하기 및 df 변환
        column_sums = subset_date.iloc[:,1:25].sum()
        column_sums_df = pd.DataFrame(column_sums).transpose()
        # 최종 df 접합 및 filtered_floating_population_tottime에 concat
        column_sums_df = pd.concat([date_df, column_sums_df], axis=1)
        concat_df = pd.concat([concat_df, column_sums_df])
    else:
        # 각 연도 별로 평균계산 및 df 변환 -> 월별 유동인구를 평균내었기 때문에, 최종값은 해당 격자의 월별 평균 유동인구가 됨.
        column_means_2020 = concat_df[concat_df['연도'].str.startswith('2020')].iloc[:,1:25].mean()
        date_df_2020 = pd.DataFrame({'연도' : ['2020']})
        column_means_2020_df = pd.DataFrame(column_means_2020).transpose()
        tot_df_2020 = pd.concat([date_df_2020, column_means_2020_df, grid_df], axis=1)
        
        column_means_2021 = concat_df[concat_df['연도'].str.startswith('2021')].iloc[:,1:25].mean()
        date_df_2021 = pd.DataFrame({'연도' : ['2021']})
        column_means_2021_df = pd.DataFrame(column_means_2021).transpose()
        tot_df_2021 = pd.concat([date_df_2021, column_means_2021_df, grid_df], axis=1)
        
        column_means_2022 = concat_df[concat_df['연도'].str.startswith('2022')].iloc[:,1:25].mean()
        date_df_2022 = pd.DataFrame({'연도' : ['2022']})
        column_means_2022_df = pd.DataFrame(column_means_2022).transpose()
        tot_df_2022 = pd.concat([date_df_2022, column_means_2022_df, grid_df], axis=1)
        
        tot_df = pd.concat([tot_df_2020, tot_df_2021, tot_df_2022])
        filtered_floating_population_tottime = pd.concat([filtered_floating_population_tottime, tot_df])

filtered_floating_population_tottime

100%|██████████| 156/156 [00:17<00:00,  8.82it/s]


,연도,00시,01시,02시,03시,04시,05시,06시,07시,08시,09시,10시,11시,12시,13시,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시,geometry
0,2020,0.743333,0.562500,0.410000,0.353333,0.397500,1.048333,2.364167,4.010000,8.285833,7.914167,7.829167,8.085833,7.725000,8.182500,7.910833,7.733333,7.835000,7.680000,7.585000,5.047500,3.394167,2.910000,2.007500,1.116667,"POLYGON ((127.4921705012675 36.62987958832503,..."
0,2021,0.538333,0.260000,0.179167,0.164167,0.203333,0.754167,1.924167,3.224167,7.108333,6.198333,6.260000,6.260833,6.111667,6.353333,6.152500,6.005000,5.910000,5.942500,5.734167,3.918333,2.715833,2.261667,1.568333,0.850000,"POLYGON ((127.4921705012675 36.62987958832503,..."
0,2022,0.789167,0.490833,0.348333,0.264167,0.295833,0.898333,2.642500,4.105000,8.075833,7.315000,7.259167,7.114167,7.075000,7.212500,7.063333,6.706667,6.917500,7.160000,6.942500,4.810000,3.466667,2.979167,2.040000,1.154167,"POLYGON ((127.4921705012675 36.62987958832503,..."
0,2020,32.125833,32.169167,22.200000,19.482500,18.862500,22.098333,36.477500,71.368333,129.768333,98.670833,88.459167,91.758333,92.458333,100.795000,98.803333,97.835833,99.687500,103.488333,113.209167,82.256667,65.828333,60.382500,50.210833,38.526667,POLYGON ((127.48881291670402 36.64430329023720...
0,2021,36.144167,33.334167,26.167500,24.463333,25.083333,32.190000,51.613333,89.057500,158.890833,116.545000,103.617500,109.870833,112.969167,123.925833,117.563333,118.200833,120.300000,127.003333,134.125833,98.308333,77.848333,70.857500,57.137500,42.739167,POLYGON ((127.48881291670402 36.64430329023720...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2021,0.841667,0.619167,0.405000,0.320000,0.390000,0.817500,1.874167,6.427500,12.013333,8.771667,8.197500,8.844167,9.032500,10.515000,10.086667,9.842500,9.745000,10.010833,10.586667,6.562500,4.454167,3.580000,2.319167,1.331667,POLYGON ((127.49105137605241 36.63528850233756...
0,2022,1.040000,1.258333,0.778333,0.647500,0.847500,1.497500,2.832500,7.814167,14.133333,10.580833,9.704167,10.586667,11.336667,13.072500,12.420000,12.367500,13.441667,16.997500,17.181667,10.004167,5.725000,4.548333,2.941667,1.709167,POLYGON ((127.49105137605241 36.63528850233756...
0,2020,0.433333,0.488333,0.274167,0.218333,0.258333,0.398333,0.830000,1.815000,3.665000,2.813333,2.498333,2.574167,2.555833,2.848333,3.021667,3.005000,3.192500,3.504167,4.297500,2.600833,1.927500,1.568333,1.130000,0.678333,POLYGON ((127.48433936092893 36.63799230366292...
0,2021,0.295833,0.298333,0.172500,0.138333,0.176667,0.314167,0.671667,1.597500,3.267500,2.410000,2.188333,2.259167,2.345833,2.602500,2.619167,2.620000,2.711667,3.030000,3.504167,2.068333,1.499167,1.231667,0.896667,0.484167,POLYGON ((127.48433936092893 36.63799230366292...
